# Scenario 2 (High Density) [50 Repetitions]

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product, chain, combinations
from scipy import stats
from IPython.display import display, HTML
%matplotlib inline

def parse_if_number(s):
    try: return float(s)
    except: return True if s=="true" else False if s=="false" else s if s else None

def parse_ndarray(s):
    return np.fromstring(s, sep=' ') if s else None

## Config

In [2]:
inputFile = '2kr-scalars-50rep.csv'
repetitionsCount = -1 # -1 = auto-detect
factors = ['R', 'T', 'm', 'D']

# Filter scalars
scalarsFilter = ['Floorplan.userCount', 'Floorplan.coveredUsers:sum', 'Floorplan.collisions:sum', 'Floorplan.activityTime:last']
# Performance indexes
perfIndexes = [
    ('coveredUsersPercent', 'percentage of covered users'),
    ('Floorplan.collisions:sum', 'total number of collisions'),
    ('Floorplan.activityTime:last', 'total broadcast time'),
]

## Load scalars

In [3]:
df = pd.read_csv('exported_data/' + inputFile, converters = {
    'attrvalue': parse_if_number,
    'binedges': parse_ndarray,
    'binvalues': parse_ndarray,
    'vectime': parse_ndarray,
    'vecvalue': parse_ndarray,
})

In [4]:
if repetitionsCount <= 0: # auto-detect
    repetitionsCount = int(df[df.attrname == 'repetition']['attrvalue'].max()) + 1
print('Repetitions:', repetitionsCount)

# Computed
factorsCount = len(factors)
configsCount = 2**factorsCount
totalSims = configsCount*repetitionsCount


display(HTML("<style>div.output_scroll { height: auto; max-height: 48em; }</style>"))
pd.set_option('display.max_rows', totalSims)
pd.set_option('display.max_columns', 2*configsCount*repetitionsCount+1)

Repetitions: 50


In [5]:
scalars = df[(df.type == 'scalar') | ((df.type == 'itervar') & (df.attrname != 'TO')) | ((df.type == 'param') & (df.attrname == 'Floorplan.userCount')) | ((df.type == 'runattr') & (df.attrname == 'repetition'))]
scalars = scalars.assign(qname = scalars.attrname.combine_first(scalars.module + '.' + scalars.name))
scalars.value = scalars.value.combine_first(scalars.attrvalue.astype('float64'))
scalars_wide = scalars.pivot_table(index=['run'], columns='qname', values='value')
scalars_wide.sort_values([*factors, 'repetition'], inplace=True)
count = 0
for index in scalars_wide.index:
    config = count // repetitionsCount
    scalars_wide.loc[index, 'config'] = config
    count += 1
scalars_wide = scalars_wide[['config', 'repetition', *factors, *scalarsFilter]]

In [6]:
# coverage
scalars_wide['coveredUsersPercent'] = scalars_wide['Floorplan.coveredUsers:sum'] / (scalars_wide['Floorplan.userCount'] - 1)

## 2kr

In [7]:
combinedFactors = list(chain(*[list(map(''.join, combinations(''.join(factors), i))) for i in range(1, factorsCount+1)]))
cols = ['exp', 'I'] + combinedFactors
data = [[e, 1] + list(chain(*[list(map(np.prod, combinations(r, i))) for i in range(1, len(r)+1)])) for r,e in zip([list(x) for x in product([-1, 1], repeat=factorsCount)], range(0, configsCount))]
factorsMatrix = pd.DataFrame.from_records(data, columns=cols, index='exp')
for perfIndex, perfIndexDesc in perfIndexes:
    #print("2^k*r analysis for the", perfIndexDesc)
    display(HTML('<h3>2^k*r analysis for the ' + perfIndexDesc + '</h3>'))
    cols = ['exp'] + ['y' + str(i) for i in range(0, repetitionsCount)]
    data = [[i] + scalars_wide[scalars_wide.config == i][perfIndex].values.tolist() for i in range(0, configsCount)]
    ysMatrix = pd.DataFrame.from_records(data, columns=cols, index='exp')
    ysMatrix['mean'] = ysMatrix.mean(axis=1)
    for r in range(0, repetitionsCount):
        ysMatrix['e' + str(r)] = ysMatrix['y' + str(r)] - ysMatrix['mean']

    matrix = pd.concat([factorsMatrix, ysMatrix], axis=1)

    computeCols = ['idx', 'I'] + combinedFactors + ['e' + str(r) for r in range(0, repetitionsCount)]
    data = [['Sum'], ['Mean (Qi)'], ['SSQ']]
    SSE = 0
    for col in computeCols[1:]:
        sumvalue = np.sum(matrix[col].values * matrix['mean'].values)
        data[0].append(sumvalue)
        data[1].append(sumvalue / configsCount)
        if col in ['e' + str(r) for r in range(0, repetitionsCount)]:
            ssq = np.sum(matrix[col].values * matrix[col].values)
            data[2].append(ssq)
            SSE += ssq
        else:
            data[2].append('')
    computeMatrix = pd.DataFrame.from_records(data, columns=computeCols, index='idx')

    data = [['(2^k)*r*(qi)^2'], ['Variation'], ['Variation%']]
    SST = SSE
    for col in combinedFactors:
        qi = computeMatrix.loc['Mean (Qi)', col]
        r2kqi2 = configsCount*repetitionsCount*(qi**2)
        data[0].append(r2kqi2)
        SST += r2kqi2
    data[0].append(SSE)
    idx = 1
    for col in combinedFactors:
        r2kqi2 = data[0][idx]
        var = r2kqi2 / SST
        data[1].append(var)
        data[2].append('{:.2%}'.format(var))
        idx += 1
    data[1].append(SST)
    data[2].append('{:.2%}'.format(SSE/SST))
    resultMatrix = pd.DataFrame.from_records(data, columns=['idx'] + combinedFactors + ['SSE/SST'], index='idx')

    display(matrix)
    display(computeMatrix)
    display(resultMatrix)

    print('SSE =', SSE)
    print('SST =', SST)
    print('Unexplained Variation =', '{:.2%}'.format(SSE/SST))
    print('####################################')
    print()

,I,R,T,m,D,RT,Rm,RD,Tm,TD,mD,RTm,RTD,RmD,TmD,RTmD,y0,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33,y34,y35,y36,y37,y38,y39,y40,y41,y42,y43,y44,y45,y46,y47,y48,y49,mean,e0,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,e11,e12,e13,e14,e15,e16,e17,e18,e19,e20,e21,e22,e23,e24,e25,e26,e27,e28,e29,e30,e31,e32,e33,e34,e35,e36,e37,e38,e39,e40,e41,e42,e43,e44,e45,e46,e47,e48,e49
exp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,-1,-1,-1,-1,1,1,1,1,1,1,-1,-1,-1,-1,1,0.845691,0.519038,0.807615,0.895792,0.903808,0.895792,0.837675,0.891784,0.915832,0.845691,0.745491,0.903808,0.955912,0.797595,0.945892,0.941884,0.867735,0.921844,0.925852,0.865731,0.879760,0.867735,0.907816,0.881764,0.865731,0.827655,0.891784,0.737475,0.462926,0.909820,0.899800,0.917836,0.919840,0.817635,0.793587,0.799599,0.899800,0.875752,0.835671,0.887776,0.813627,0.905812,0.903808,0.885772,0.827655,0.867735,0.867735,0.877756,0.923848,0.847695,0.856633,-0.010942,-0.337595,-0.049018,0.039158,0.047174,0.039158,-0.018958,0.035150,0.059198,-0.010942,-0.111142,0.047174,0.099279,-0.059038,0.089259,0.085251,0.011102,0.065210,0.069218,0.009098,0.023126,0.011102,0.051182,0.025130,0.009098,-0.028978,0.035150,-0.119158,-0.393707,0.053186,0.043166,0.061202,0.063206,-0.038998,-0.063046,-0.057034,0.043166,0.019118,-0.020962,0.031142,-0.043006,0.049178,0.047174,0.029138,-0.028978,0.011102,0.011102,0.021122,0.067214,-0.008938
1,1,-1,-1,-1,1,1,1,-1,1,-1,-1,-1,1,1,1,-1,0.983968,0.953908,0.991984,0.963928,0.965932,0.973948,0.957916,0.979960,0.989980,0.995992,0.931864,0.983968,0.991984,0.989980,0.989980,0.995992,0.947896,0.985972,0.961924,0.991984,0.997996,0.993988,0.989980,0.971944,0.939880,0.983968,0.997996,0.987976,0.995992,0.973948,0.995992,0.965932,0.981964,0.967936,0.957916,0.975952,0.997996,0.987976,0.973948,0.963928,0.987976,0.995992,0.985972,0.953908,0.993988,0.989980,0.951904,0.997996,0.995992,0.997996,0.979679,0.004289,-0.025772,0.012305,-0.015752,-0.013747,-0.005731,-0.021764,0.000281,0.010301,0.016313,-0.047816,0.004289,0.012305,0.010301,0.010301,0.016313,-0.031784,0.006293,-0.017756,0.012305,0.018317,0.014309,0.010301,-0.007735,-0.039800,0.004289,0.018317,0.008297,0.016313,-0.005731,0.016313,-0.013747,0.002285,-0.011743,-0.021764,-0.003727,0.018317,0.008297,-0.005731,-0.015752,0.008297,0.016313,0.006293,-0.025772,0.014309,0.010301,-0.027776,0.018317,0.016313,0.018317
2,1,-1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,1,-1,0.977956,0.955912,0.977956,0.979960,0.979960,0.973948,0.983968,0.965932,0.975952,0.975952,0.963928,0.953908,0.953908,0.975952,0.983968,0.951904,0.967936,0.967936,0.983968,0.973948,0.977956,0.985972,0.975952,0.983968,0.973948,0.977956,0.983968,0.939880,0.969940,0.967936,0.963928,0.973948,0.963928,0.963928,0.979960,0.971944,0.967936,0.973948,0.961924,0.971944,0.973948,0.961924,0.979960,0.967936,0.991984,0.987976,0.973948,0.959920,0.961924,0.971944,0.971743,0.006212,-0.015832,0.006212,0.008216,0.008216,0.002204,0.012224,-0.005812,0.004208,0.004208,-0.007816,-0.017836,-0.017836,0.004208,0.012224,-0.019840,-0.003808,-0.003808,0.012224,0.002204,0.006212,0.014228,0.004208,0.012224,0.002204,0.006212,0.012224,-0.031864,-0.001804,-0.003808,-0.007816,0.002204,-0.007816,-0.007816,0.008216,0.000200,-0.003808,0.002204,-0.009820,0.000200,0.002204,-0.009820,0.008216,-0.003808,0.020240,0.016232,0.002204,-0.011824,-0.009820,0.000200
3,1,-1,-1,1,1,1,-1,-1,-1,-1,1,1,1,-1,-1,1,0.997996,0.995992,0.997996,0.991984,0.993988,0.995992,0.997996,0.977956,0.995992,0.993988,0.997996,1.000000,0.993988,1.000000,0.997996,1.000000,0.997996,0.991984,0.997996,0.997996,0.997996,0.995992,0.993988,1.000000,0.993988,0.997996,1.000000,0.995992,0.991984,1.000000,0.997996,1.000000,0.997996,1.000000,0.993988,0.991984,0.995992,1.000000,1.000000,1.000000,1.000000,0.995992,0.991984,0.997996,0.997996,0.995992,0.995992,0.995992,0.989980,1.000000,0.996473,0.001523,-0.000481,0.001523,-0.004

,I,R,T,m,D,RT,Rm,RD,Tm,TD,mD,RTm,RTD,RmD,TmD,RTmD,e0,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,e11,e12,e13,e14,e15,e16,e17,e18,e19,e20,e21,e22,e23,e24,e25,e26,e27,e28,e29,e30,e31,e32,e33,e34,e35,e36,e37,e38,e39,e40,e41,e42,e43,e44,e45,e46,e47,e48,e49
idx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Sum,14.211904,-0.822325,0.200842,0.609178,0.631543,0.384729,-0.099599,0.234749,0.203567,0.036593,-0.406212,0.022405,0.230902,-0.235631,0.236513,0.013828,0.125356,-0.961781,0.102082,0.248668,0.099518,0.003699,0.340291,0.279670,-0.158428,0.474016,-0.791374,0.063779,0.304103,-0.111596,0.053106,0.377281,0.045881,-0.097641,0.111669,0.268151,-0.630431,0.280570,-0.134602,0.301398,-0.313019,0.000373,0.058062,-0.100434,-0.156645,0.151135,0.442261,0.095444,0.262077,-0.390450,-0.609932,-0.031540,0.332202,-0.026070,0.061383,-0.080362,0.138567,0.297107,0.335392,-0.606237,-0.285278,0.042093,-0.628644,0.240061,0.039651,0.139422
Mean (Qi),0.888244,-0.051395,0.012553,0.038074,0.039471,0.024046,-0.006225,0.014672,0.012723,0.002287,-0.025388,0.0014,0.014431,-0.014727,0.014782,0.000864,0.007835,-0.060111,0.006380,0.015542,0.006220,0.000231,0.021268,0.017479,-0.009902,0.029626,-0.049461,0.003986,0.019006,-0.006975,0.003319,0.023580,0.002868,-0.006103,0.006979,0.016759,-0.039402,0.017536,-0.008413,0.018837,-0.019564,0.000023,0.003629,-0.006277,-0.009790,0.009446,0.027641,0.005965,0.016380,-0.024403,-0.038121,-0.001971,0.020763,-0.001629,0.003836,-0.005023,0.008660,0.018569,0.020962,-0.037890,-0.017830,0.002631,-0.039290,0.015004,0.002478,0.008714
SSQ,,,,,,,,,,,,,,,,,0.076421,0.542225,0.045282,0.039507,0.045560,0.029189,0.036329,0.068911,0.165259,0.065166,0.638738,0.045455,0.054150,0.046929,0.024576,0.037907,0.102082,0.115313,0.050140,0.067179,0.959247,0.065727,0.073858,0.048838,0.095841,0.057517,0.033684,0.077500,0.208689,0.031715,0.083322,0.076040,0.044701,0.076021,0.548346,0.044625,0.028385,0.036961,0.062198,0.032173,0.057569,0.041055,0.101488,0.225838,0.347036,0.053004,0.691243,0.044726,0.103703,0.071481


,R,T,m,D,RT,Rm,RD,Tm,TD,mD,RTm,RTD,RmD,TmD,RTmD,SSE/SST
idx,,,,,,,,,,,,,,,,
(2^k)*r*(qi)^2,2.113181,0.126054,1.159682,1.246396,0.462552,0.031,0.17221,0.129499,0.004185,0.515652,0.001569,0.166611,0.173507,0.174808,0.000598,6.718851
Variation,0.160134,0.009552,0.087879,0.09445,0.035052,0.002349,0.01305,0.009813,0.000317,0.039075,0.000119,0.012626,0.013148,0.013247,0.000045,13.196354
Variation%,16.01%,0.96%,8.79%,9.45%,3.51%,0.23%,1.30%,0.98%,0.03%,3.91%,0.01%,1.26%,1.31%,1.32%,0.00%,50.91%


SSE = 6.718851490556262
SST = 13.19635378472375
Unexplained Variation = 50.91%
####################################



,I,R,T,m,D,RT,Rm,RD,Tm,TD,mD,RTm,RTD,RmD,TmD,RTmD,y0,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33,y34,y35,y36,y37,y38,y39,y40,y41,y42,y43,y44,y45,y46,y47,y48,y49,mean,e0,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,e11,e12,e13,e14,e15,e16,e17,e18,e19,e20,e21,e22,e23,e24,e25,e26,e27,e28,e29,e30,e31,e32,e33,e34,e35,e36,e37,e38,e39,e40,e41,e42,e43,e44,e45,e46,e47,e48,e49
exp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,-1,-1,-1,-1,1,1,1,1,1,1,-1,-1,-1,-1,1,267.0,228.0,306.0,302.0,305.0,360.0,261.0,304.0,283.0,266.0,309.0,315.0,280.0,277.0,299.0,366.0,308.0,303.0,295.0,306.0,248.0,314.0,327.0,297.0,299.0,325.0,290.0,241.0,181.0,297.0,332.0,283.0,306.0,363.0,285.0,299.0,316.0,330.0,280.0,248.0,323.0,320.0,336.0,348.0,314.0,334.0,364.0,318.0,326.0,362.0,302.92,-35.92,-74.92,3.08,-0.92,2.08,57.08,-41.92,1.08,-19.92,-36.92,6.08,12.08,-22.92,-25.92,-3.92,63.08,5.08,0.08,-7.92,3.08,-54.92,11.08,24.08,-5.92,-3.92,22.08,-12.92,-61.92,-121.92,-5.92,29.08,-19.92,3.08,60.08,-17.92,-3.92,13.08,27.08,-22.92,-54.92,20.08,17.08,33.08,45.08,11.08,31.08,61.08,15.08,23.08,59.08
1,1,-1,-1,-1,1,1,1,-1,1,-1,-1,-1,1,1,1,-1,172.0,189.0,213.0,217.0,212.0,239.0,221.0,165.0,210.0,148.0,220.0,180.0,141.0,190.0,218.0,169.0,208.0,200.0,203.0,172.0,194.0,173.0,178.0,203.0,177.0,168.0,165.0,176.0,183.0,228.0,196.0,224.0,231.0,130.0,153.0,195.0,161.0,220.0,180.0,188.0,162.0,174.0,175.0,206.0,203.0,164.0,171.0,206.0,172.0,205.0,188.96,-16.96,0.04,24.04,28.04,23.04,50.04,32.04,-23.96,21.04,-40.96,31.04,-8.96,-47.96,1.04,29.04,-19.96,19.04,11.04,14.04,-16.96,5.04,-15.96,-10.96,14.04,-11.96,-20.96,-23.96,-12.96,-5.96,39.04,7.04,35.04,42.04,-58.96,-35.96,6.04,-27.96,31.04,-8.96,-0.96,-26.96,-14.96,-13.96,17.04,14.04,-24.96,-17.96,17.04,-16.96,16.04
2,1,-1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,1,-1,639.0,653.0,643.0,617.0,619.0,642.0,619.0,669.0,659.0,679.0,603.0,638.0,715.0,610.0,645.0,594.0,672.0,633.0,701.0,651.0,713.0,647.0,678.0,660.0,672.0,642.0,643.0,637.0,601.0,637.0,647.0,631.0,634.0,639.0,655.0,630.0,664.0,658.0,671.0,605.0,673.0,605.0,729.0,637.0,671.0,605.0,656.0,589.0,637.0,613.0,645.60,-6.60,7.40,-2.60,-28.60,-26.60,-3.60,-26.60,23.40,13.40,33.40,-42.60,-7.60,69.40,-35.60,-0.60,-51.60,26.40,-12.60,55.40,5.40,67.40,1.40,32.40,14.40,26.40,-3.60,-2.60,-8.60,-44.60,-8.60,1.40,-14.60,-11.60,-6.60,9.40,-15.60,18.40,12.40,25.40,-40.60,27.40,-40.60,83.40,-8.60,25.40,-40.60,10.40,-56.60,-8.60,-32.60
3,1,-1,-1,1,1,1,-1,-1,-1,-1,1,1,1,-1,-1,1,351.0,340.0,310.0,316.0,371.0,355.0,374.0,324.0,291.0,356.0,325.0,321.0,314.0,348.0,314.0,355.0,386.0,344.0,308.0,378.0,385.0,296.0,279.0,345.0,294.0,327.0,332.0,316.0,372.0,379.0,275.0,335.0,289.0,327.0,434.0,364.0,295.0,375.0,361.0,345.0,311.0,300.0,320.0,346.0,332.0,403.0,334.0,326.0,341.0,322.0,336.82,14.18,3.18,-26.82,-20.82,34.18,18.18,37.18,-12.82,-45.82,19.18,-11.82,-15.82,-22.82,11.18,-22.82,18.18,49.18,7.18,-28.82,41.18,48.18,-40.82,-57.82,8.18,-42.82,-9.82,-4.82,-20.82,35.18,42.18,-61.82,-1.82,-47.82,-9.82,97.18,27.18,-41.82,38.18,24.18,8.18,-25.82,-36.82,-16.82,9.18,-4.82,66.18,-2.82,-10.82,4.18,-14.82
4,1,-1,1,-1,-1,-1,1,1,-1,-1,1,1,1,-1,1,-1,286.0,366.0,271.0,326.0,278.0,302.0,345.0,306.0,265.0,320.0,318.0,274.0,367.0,286.0,338.0,293.0,329.0,273.0,360.0,347.0,330.0,298.0,356.0,323.0,235.0,238.0,272.0,302.0,300.0,354.0,334.0,298.0,329.0,290.0,288.0,213.0,272.0,341.0,323.0,278.0,345.0,315.0,324.0,121.0,352.0,260.0,277.0,287.0,371.0,335.0,304.22,-18.22,61.78,-33.22,21.78,-26.22,-2.22,40.78,1.78,-39.22,15.78,13.78,-30.22,62.78,-18.22,33.78,-11.22,24.78,-31.22,55.78,42.78,25.78,-6.22,51.78,18.78,-69.22,-66.22,-32.22,-2.22,-4.22,49.78,29.78,-6.22,24.78,-14.22,-16.22,-91.22,-32.22,36.78,18.78,-26.22,40.78,10.78,19.78,-183.22,47.78,-44.22,-27.22,-17.22,66.78,30.78
5,1,-1,1,-1,1,-1,1,-1,-1,1,-1,1,-1,1,-1,1,119.0,59.0,130.0,126.0,153.0,139.0,151.0,148.0,116.0,169.0,7.0,155.0,139.0,126.0,129.0,

...

SSE = 15112317.68
SST = 337501200.8749999
Unexplained Variation = 4.48%
####################################



,I,R,T,m,D,RT,Rm,RD,Tm,TD,mD,RTm,RTD,RmD,TmD,RTmD,y0,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33,y34,y35,y36,y37,y38,y39,y40,y41,y42,y43,y44,y45,y46,y47,y48,y49,mean,e0,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,e11,e12,e13,e14,e15,e16,e17,e18,e19,e20,e21,e22,e23,e24,e25,e26,e27,e28,e29,e30,e31,e32,e33,e34,e35,e36,e37,e38,e39,e40,e41,e42,e43,e44,e45,e46,e47,e48,e49
exp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,-1,-1,-1,-1,1,1,1,1,1,1,-1,-1,-1,-1,1,35.0,24.0,43.0,32.0,40.0,46.0,60.0,33.0,37.0,46.0,39.0,39.0,43.0,33.0,45.0,53.0,39.0,41.0,49.0,56.0,37.0,43.0,49.0,62.0,39.0,42.0,36.0,43.0,23.0,34.0,31.0,50.0,52.0,34.0,43.0,52.0,64.0,27.0,33.0,56.0,62.0,41.0,47.0,33.0,52.0,56.0,36.0,41.0,39.0,39.0,42.58,-7.58,-18.58,0.42,-10.58,-2.58,3.42,17.42,-9.58,-5.58,3.42,-3.58,-3.58,0.42,-9.58,2.42,10.42,-3.58,-1.58,6.42,13.42,-5.58,0.42,6.42,19.42,-3.58,-0.58,-6.58,0.42,-19.58,-8.58,-11.58,7.42,9.42,-8.58,0.42,9.42,21.42,-15.58,-9.58,13.42,19.42,-1.58,4.42,-9.58,9.42,13.42,-6.58,-1.58,-3.58,-3.58
1,1,-1,-1,-1,1,1,1,-1,1,-1,-1,-1,1,1,1,-1,54.0,47.0,43.0,54.0,52.0,56.0,59.0,54.0,54.0,57.0,42.0,78.0,39.0,52.0,55.0,80.0,39.0,55.0,42.0,50.0,56.0,58.0,52.0,56.0,63.0,54.0,41.0,65.0,56.0,61.0,55.0,45.0,58.0,50.0,66.0,59.0,36.0,52.0,44.0,58.0,63.0,57.0,63.0,45.0,53.0,62.0,65.0,46.0,73.0,111.0,55.70,-1.70,-8.70,-12.70,-1.70,-3.70,0.30,3.30,-1.70,-1.70,1.30,-13.70,22.30,-16.70,-3.70,-0.70,24.30,-16.70,-0.70,-13.70,-5.70,0.30,2.30,-3.70,0.30,7.30,-1.70,-14.70,9.30,0.30,5.30,-0.70,-10.70,2.30,-5.70,10.30,3.30,-19.70,-3.70,-11.70,2.30,7.30,1.30,7.30,-10.70,-2.70,6.30,9.30,-9.70,17.30,55.30
2,1,-1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,1,-1,29.0,36.0,49.0,37.0,35.0,37.0,34.0,36.0,42.0,34.0,34.0,29.0,33.0,28.0,33.0,34.0,27.0,32.0,29.0,35.0,32.0,43.0,40.0,44.0,31.0,32.0,28.0,36.0,34.0,31.0,43.0,32.0,31.0,37.0,36.0,39.0,35.0,41.0,33.0,41.0,42.0,36.0,42.0,39.0,37.0,31.0,30.0,30.0,37.0,31.0,35.14,-6.14,0.86,13.86,1.86,-0.14,1.86,-1.14,0.86,6.86,-1.14,-1.14,-6.14,-2.14,-7.14,-2.14,-1.14,-8.14,-3.14,-6.14,-0.14,-3.14,7.86,4.86,8.86,-4.14,-3.14,-7.14,0.86,-1.14,-4.14,7.86,-3.14,-4.14,1.86,0.86,3.86,-0.14,5.86,-2.14,5.86,6.86,0.86,6.86,3.86,1.86,-4.14,-5.14,-5.14,1.86,-4.14
3,1,-1,-1,1,1,1,-1,-1,-1,-1,1,1,1,-1,-1,1,49.0,48.0,39.0,47.0,37.0,40.0,52.0,33.0,53.0,36.0,60.0,53.0,57.0,38.0,44.0,65.0,50.0,55.0,52.0,40.0,49.0,54.0,47.0,42.0,55.0,50.0,38.0,42.0,48.0,54.0,48.0,41.0,40.0,38.0,49.0,47.0,40.0,50.0,38.0,50.0,47.0,57.0,52.0,54.0,58.0,54.0,46.0,49.0,49.0,63.0,47.94,1.06,0.06,-8.94,-0.94,-10.94,-7.94,4.06,-14.94,5.06,-11.94,12.06,5.06,9.06,-9.94,-3.94,17.06,2.06,7.06,4.06,-7.94,1.06,6.06,-0.94,-5.94,7.06,2.06,-9.94,-5.94,0.06,6.06,0.06,-6.94,-7.94,-9.94,1.06,-0.94,-7.94,2.06,-9.94,2.06,-0.94,9.06,4.06,6.06,10.06,6.06,-1.94,1.06,1.06,15.06
4,1,-1,1,-1,-1,-1,1,1,-1,-1,1,1,1,-1,1,-1,105.0,90.0,121.0,118.0,125.0,104.0,151.0,153.0,92.0,109.0,121.0,132.0,85.0,98.0,92.0,72.0,84.0,76.0,105.0,101.0,64.0,89.0,65.0,89.0,79.0,97.0,84.0,103.0,88.0,111.0,105.0,124.0,104.0,76.0,120.0,116.0,90.0,119.0,81.0,120.0,112.0,84.0,122.0,63.0,119.0,92.0,93.0,98.0,95.0,131.0,101.34,3.66,-11.34,19.66,16.66,23.66,2.66,49.66,51.66,-9.34,7.66,19.66,30.66,-16.34,-3.34,-9.34,-29.34,-17.34,-25.34,3.66,-0.34,-37.34,-12.34,-36.34,-12.34,-22.34,-4.34,-17.34,1.66,-13.34,9.66,3.66,22.66,2.66,-25.34,18.66,14.66,-11.34,17.66,-20.34,18.66,10.66,-17.34,20.66,-38.34,17.66,-9.34,-8.34,-3.34,-6.34,29.66
5,1,-1,1,-1,1,-1,1,-1,-1,1,-1,1,-1,1,-1,1,154.0,83.0,143.0,109.0,170.0,144.0,157.0,120.0,145.0,133.0,29.0,155.0,178.0,140.0,118.0,102.0,126.0,139.0,136.0,113.0,155.0,140.0,87.0,187.0,141.0,154.0,134.0,126.0,143.0,191.0,159.0,180.0,179.0,139.0,31.0,131.0,161.0,92.0,127.0,95.0,112.0,157.0,117.0,114.0,142.0,148.0,26.0,111.0,97.0,122.0,129.84,24.16,-46.84,13.16,-20.84,40.16,14.16,27.16,-9.84,15.16,3.16,-100.84,25.16,48.16,10.16,-11.84,-27.84,-3.84,9.16,6.16,-16.84,25.16,10.16,-4

,I,R,T,m,D,RT,Rm,RD,Tm,TD,mD,RTm,RTD,RmD,TmD,RTmD,e0,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,e11,e12,e13,e14,e15,e16,e17,e18,e19,e20,e21,e22,e23,e24,e25,e26,e27,e28,e29,e30,e31,e32,e33,e34,e35,e36,e37,e38,e39,e40,e41,e42,e43,e44,e45,e46,e47,e48,e49
idx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Sum,783.04,-427.6,312.8,-44.0,81.08,-172.4,63.2,-57.64,-24.72,17.88,-30.76,21.68,-17.16,-3.0,-22.84,3.64,-772.10320,-7454.6432,4413.65680,-1727.02320,5243.21680,-1377.2032,6402.5168,882.17680,1193.6768,-455.22320,-11630.8832,10952.1968,2083.21680,-868.18320,-738.1232,-5485.30320,-2187.14320,-2827.2832,3631.57680,-1455.54320,-8169.8832,3041.69680,-9362.8432,8309.57680,791.8768,3007.7968,-3511.6032,-1861.1632,2027.5568,11320.49680,3555.2768,6713.89680,7521.9168,-2848.4032,-11533.4032,2863.5968,2006.3168,-3783.1632,-773.1632,-271.5232,3608.6368,4806.2768,2602.85680,-5927.46320,5308.6368,3577.61680,-16192.6032,-2605.6432,-6313.6832,4266.93680
Mean (Qi),48.94,-26.725,19.55,-2.75,5.0675,-10.775,3.95,-3.6025,-1.545,1.1175,-1.9225,1.355,-1.0725,-0.1875,-1.4275,0.2275,-48.25645,-465.9152,275.85355,-107.93895,327.70105,-86.0752,400.1573,55.13605,74.6048,-28.45145,-726.9302,684.5123,130.20105,-54.26145,-46.1327,-342.83145,-136.69645,-176.7052,226.97355,-90.97145,-510.6177,190.10605,-585.1777,519.34855,49.4923,187.9873,-219.4752,-116.3227,126.7223,707.53105,222.2048,419.61855,470.1198,-178.0252,-720.8377,178.9748,125.3948,-236.4477,-48.3227,-16.9702,225.5398,300.3923,162.67855,-370.46645,331.7898,223.60105,-1012.0377,-162.8527,-394.6052,266.68355
SSQ,,,,,,,,,,,,,,,,,1342.52320,3372.6032,1601.00320,907.36320,3234.88320,991.7232,4404.2832,3375.96320,616.9632,1329.76320,10945.0832,3043.9232,3257.88320,1475.68320,684.5632,3095.92320,993.60320,957.8832,890.16320,1225.40320,8962.0832,963.92320,4736.0032,4184.16320,1351.5632,927.7232,1864.5232,828.6432,1258.2032,4536.32320,2060.7632,3706.52320,2974.4832,1094.1232,10463.1232,453.1232,2696.6832,2538.6432,1672.6432,2061.3632,1914.0432,1772.7632,1045.60320,2192.24320,1465.0432,840.08320,11352.5232,708.6032,2374.6832,4682.44320


,R,T,m,D,RT,Rm,RD,Tm,TD,mD,RTm,RTD,RmD,TmD,RTmD,SSE/SST
idx,,,,,,,,,,,,,,,,
(2^k)*r*(qi)^2,571380.5,305762.0,6050.0,20543.645,92880.5,12482.0,10382.405,1909.62,999.045,2956.805,1468.82,920.205,28.125,1630.205,41.405,135429.84
Variation,0.490512,0.262487,0.005194,0.017636,0.079735,0.010715,0.008913,0.001639,0.000858,0.002538,0.001261,0.00079,0.000024,0.001399,0.000036,1164865.12
Variation%,49.05%,26.25%,0.52%,1.76%,7.97%,1.07%,0.89%,0.16%,0.09%,0.25%,0.13%,0.08%,0.00%,0.14%,0.00%,11.63%


SSE = 135429.84
SST = 1164865.12
Unexplained Variation = 11.63%
####################################

